In [1]:
!pip install google-api-python-client

In [2]:
!pip install google-api-python-client

In [3]:
pip install langdetect

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pytube

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install --upgrade google-auth-oauthlib google-auth-httplib2

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install emoji==1.7

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install text2emotion

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
from googleapiclient.discovery import build
import numpy as np
from pytube import Playlist,extract
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import statistics
from langdetect import detect
import emoji
import text2emotion as te
import re

[nltk_data] Downloading package stopwords to C:\Users\Ivan
[nltk_data]     Tai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Ivan
[nltk_data]     Tai\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Ivan
[nltk_data]     Tai\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
#https://www.youtube.com/watch?v=SwSbnmqk3zY

In [10]:
df_sad_song=pd.read_csv("sad_song_ytvideo_statistics.csv")

In [ ]:
secrets_filename = 'secret_keys'
api_keys = {}
with open(secrets_filename, 'r') as f:
    api_keys = json.loads(f.read())

In [ ]:
youtube=build("youtube","v3",developerKey=api_keys)

In [12]:
sia = SentimentIntensityAnalyzer()

In [13]:
text_list=[]
id_list=[]
pos_list=[]
neg_list=[]
neu_list=[]
compound_list=[]   


for id in df_sad_song["id"]:   
    try:
        results = youtube.commentThreads().list(
        part="snippet",
        videoId=id,
        textFormat="plainText",
        maxResults=100
        ).execute()
    except:
        pass
    #Get the 100 comments from video
    for item in results["items"]:
        comment = item["snippet"]["topLevelComment"]
        text = comment["snippet"]["textDisplay"]
     #preprossessing of comment text
        text = text.lower().strip()
        try: 
          language=detect(text)
        except:
           language=" "
        # append the useful text to the list, and calculate their sentiment score
        if language=="en" and len(text)<300:
           text_list.append(text)
           id_list.append(id) 
           sentiment_dict=sia.polarity_scores(text)
           pos_list.append(sentiment_dict["pos"])
           neg_list.append(sentiment_dict["neg"])
           neu_list.append(sentiment_dict["neu"])
           compound_list.append(sentiment_dict["compound"])

lists=[text_list,id_list,neg_list,neu_list,pos_list,
compound_list]
for list in lists:
    list=np.array(list) 



In [14]:
comments_dict={"id":id_list,"text":text_list,"neg":neg_list,
               "neu":neu_list,"pos":pos_list,"overall_sentiment":compound_list} 

In [15]:
df_sad_comment=pd.DataFrame(comments_dict)
df_sad_comment

,id,text,neg,neu,pos,overall_sentiment
0,Jkj36B1YuDU,"subscribe to our new channel ""7clouds music mi...",0.000,0.772,0.228,0.7777
1,Jkj36B1YuDU,powerful lyrics,0.000,0.263,0.737,0.4215
2,Jkj36B1YuDU,ok who else just listens to sad song when ur a...,0.399,0.450,0.151,-0.8271
3,Jkj36B1YuDU,same as three,0.000,1.000,0.000,0.0000
4,Jkj36B1YuDU,still here at 295 million views 2025🎉 january ...,0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...
5022,A48hOToMuRE,❤❤❤ this song makes my heart explode.,0.000,1.000,0.000,0.0000
5023,A48hOToMuRE,"can anyone explain why people sing w this 1 ""a...",0.000,1.000,0.000,0.0000
5024,A48hOToMuRE,this a classic,0.000,1.000,0.000,0.0000
5025,A48hOToMuRE,christopher extension,0.000,1.000,0.000,0.0000


In [16]:
# calculate the emotion_score(take 45 minutes to load)
def emotion_dict1(df):
    emotion_dict=te.get_emotion(df)
    emotion_score1=emotion_dict["Angry"]
    return emotion_score1

def emotion_dict2(df):
    emotion_dict=te.get_emotion(df)
    emotion_score2=emotion_dict["Surprise"]
    return emotion_score2

def emotion_dict3(df):
    emotion_dict=te.get_emotion(df)
    emotion_score3=emotion_dict["Happy"]
    return emotion_score3

def emotion_dict4(df):
    emotion_dict=te.get_emotion(df)
    emotion_score4=emotion_dict["Fear"]
    return emotion_score4

def emotion_dict5(df):
    emotion_dict=te.get_emotion(df)
    emotion_score5=emotion_dict["Sad"]
    return emotion_score5

def extract_emotion(df):
    df["Angry"]=df["text"].apply(emotion_dict1)
    df["Surprise"]=df["text"].apply(emotion_dict2)
    df["Happy"]=df["text"].apply(emotion_dict3)
    df["Fear"]=df["text"].apply(emotion_dict4)
    df["Sad"]=df["text"].apply(emotion_dict5)


extract_emotion(df=df_sad_comment)


In [50]:
def sentiment_category(score):
    if score>=0.5:  # Positive sentiment score
        return"Strongly Positive"  # High rating and positive sentiment
    elif 0.0 <= score < 0.5:
        return "Mildly Positive"
    elif -0.5 <= score < 0.0:  # Negative sentiment score
        return 'Mildly Negative'  # Low rating and negative sentiment
    else:
        return 'Strongly_Negative'  # Neutral rating and neutral sentiment

# Define a function to bucket sentiment scores into text ranges

def sentiment_range(score):    
    if score >= 0.5:
        return '0.5 to 1.0'  # Strongly positive sentiment
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'  # Mildly positive sentiment
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'  # Mildly negative sentiment
    else:
        return '-1.0 to -0.5'  # Strongly negative sentiment

In [51]:
def emotion_category(score):
    if score>=0.5:  # Positive intense emotion score
        return"intense"  # intense emotion
    elif 0.0 < score < 0.5:
        return "Mild" #mild emotion 
    else:
        return "None"
# Define a function to bucket emotion scores into text ranges

def emotion_range(score):    
    if score >= 0.5:
        return '0.5 to 1.0'  # intense emotion
    elif 0.0 < score < 0.5:
        return '0.01 to 0.49'  # Mild emotion
    else: 
        return "0.00 to 0.00"

In [52]:

#Apply sentiment categorization using both text and rating
df_sad_comment["SentimentCategory"] = df_sad_comment["overall_sentiment"].apply(sentiment_category)

# Apply sentiment bucketing to categorize scores into defined ranges
df_sad_comment['SentimentBucket'] = df_sad_comment["overall_sentiment"].apply(sentiment_range)

emotion_list=["Angry","Surprise","Happy","Fear","Sad"]

for emotion in emotion_list:
   df_sad_comment[f"{emotion} category"]=df_sad_comment[emotion].apply(emotion_category)
   df_sad_comment[f"{emotion} Buckek"]=df_sad_comment[emotion].apply(emotion_range)

df_sad_comment

,id,text,neg,neu,pos,overall_sentiment,Angry,Surprise,Happy,Fear,...,Angry category,Angry Buckek,Surprise category,Surprise Buckek,Happy category,Happy Buckek,Fear category,Fear Buckek,Sad category,Sad Buckek
0,Jkj36B1YuDU,"subscribe to our new channel ""7clouds music mi...",0.000,0.772,0.228,0.7777,0.0,0.0,1.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,intense,0.5 to 1.0,None,0.00 to 0.00,None,0.00 to 0.00
1,Jkj36B1YuDU,powerful lyrics,0.000,0.263,0.737,0.4215,0.0,0.0,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00
2,Jkj36B1YuDU,ok who else just listens to sad song when ur a...,0.399,0.450,0.151,-0.8271,0.0,0.0,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,intense,0.5 to 1.0
3,Jkj36B1YuDU,same as three,0.000,1.000,0.000,0.0000,0.0,0.0,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00
4,Jkj36B1YuDU,still here at 295 million views 2025🎉 january ...,0.000,1.000,0.000,0.0000,0.0,0.0,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5022,A48hOToMuRE,❤❤❤ this song makes my heart explode.,0.000,1.000,0.000,0.0000,0.0,0.0,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,intense,0.5 to 1.0
5023,A48hOToMuRE,"can anyone explain why people sing w this 1 ""a...",0.000,1.000,0.000,0.0000,0.0,0.0,0.0,1.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,intense,0.5 to 1.0,None,0.00 to 0.00
5024,A48hOToMuRE,this a classic,0.000,1.000,0.000,0.0000,0.0,0.0,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00
5025,A48hOToMuRE,christopher extension,0.000,1.000,0.000,0.0000,0.0,1.0,0.0,0.0,...,None,0.00 to 0.00,intense,0.5 to 1.0,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00


In [43]:
# group id with sentiment score
column_list=["neg","neu","pos","overall_sentiment","Sad","Angry","Surprise","Happy","Fear"]
df_comment_avg_score= df_sad_comment.groupby(["id"])[column_list].mean().reset_index()

# add the new column to df_rock_song
for column in column_list:
    df_sad_song[f"Avg_{column}"]=df_comment_avg_score[f"{column}"]
    df_sad_song



In [44]:
df_sad_comment[df_sad_comment["overall_sentiment"].isnull()]

,id,text,neg,neu,pos,overall_sentiment,Angry,Surprise,Happy,Fear,...,Angry category,Angry Buckek,Surprise category,Surprise Buckek,Happy category,Happy Buckek,Fear category,Fear Buckek,Sad category,Sad Buckek


In [45]:
# Apply sentiment categorization using both text and rating
df_sad_song["SentimentCategory"] = df_sad_song["Avg_overall_sentiment"].apply(sentiment_category)

# Apply sentiment bucketing to categorize scores into defined ranges
df_sad_song['SentimentBucket'] = df_sad_song["Avg_overall_sentiment"].apply(sentiment_range)

for emotion in emotion_list:
   df_sad_song[f"{emotion} category"]=df_sad_song[f"Avg_{emotion}"].apply(emotion_category)
   df_sad_song[f"{emotion} Buckek"]=df_sad_song[f"Avg_{emotion}"].apply(emotion_range)

df_sad_song

,title,id,channel_title,date,viewCount,likeCount,commentCount,musician,song_title,Year,...,Angry category,Angry Buckek,Surprise category,Surprise Buckek,Happy category,Happy Buckek,Fear category,Fear Buckek,Sad category,Sad Buckek
0,Tom Odell - Another Love (Lyrics),Jkj36B1YuDU,7clouds,2020-09-05 19:11:17,295250766,2984182,75483,Tom Odell,Another Love,2020,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
1,Benson Boone - Beautiful Things (Official Musi...,Oa_RSwwpPaA,Benson Boone,2024-01-18 23:00:13,451253770,4077315,89238,Benson Boone,Beautiful Things,2024,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
2,James Arthur - Train Wreck (Lyrics),CveANi17YfU,Lazy Fox,2020-09-08 15:40:52,265609174,2338276,58810,James Arthur,Train Wreck,2020,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
3,d4vd - Here With Me (Official Audio),B7Y4LHbpXv0,d4vd,2022-07-18 17:31:19,116828268,868085,10933,d4vd,Here With Me,2022,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
4,Miley Cyrus - Used To Be Young (Official Video),IZ3XMOdOdKM,MileyCyrusVEVO,2023-08-25 04:00:11,67690332,1281424,44587,Miley Cyrus,Used To Be Young,2023,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Billie Eilish - everything i wanted (Official ...,qCTMq7xvdXU,BillieEilishVEVO,2019-11-14 00:00:12,204177568,2957984,123869,Billie Eilish,everything i wanted,2019,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
96,Imagine Dragons - Birds (Animated Video),vOXZkm9p_zY,ImagineDragonsVEVO,2019-07-24 04:00:04,309277581,2930219,54681,Imagine Dragons,Birds,2019,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
97,Miley Cyrus - Slide Away (Official Video),rrvFv6j3-sM,MileyCyrusVEVO,2019-09-06 16:00:12,60471242,967497,33659,Miley Cyrus,Slide Away,2019,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
98,"Lady Gaga, Bradley Cooper - Shallow (from A St...",bo_efYhYU2A,LadyGagaVEVO,2018-09-27 16:30:01,1727230193,8429618,207413,"Lady Gaga, Bradley Cooper",Shallow (from A Star Is Born),2018,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49


In [46]:
df_sad_song=df_sad_song.dropna(axis=0).reset_index(drop=True)
# drop the null value again

In [53]:
df_sad_comment


,id,text,neg,neu,pos,overall_sentiment,Angry,Surprise,Happy,Fear,...,Angry category,Angry Buckek,Surprise category,Surprise Buckek,Happy category,Happy Buckek,Fear category,Fear Buckek,Sad category,Sad Buckek
0,Jkj36B1YuDU,"subscribe to our new channel ""7clouds music mi...",0.000,0.772,0.228,0.7777,0.0,0.0,1.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,intense,0.5 to 1.0,None,0.00 to 0.00,None,0.00 to 0.00
1,Jkj36B1YuDU,powerful lyrics,0.000,0.263,0.737,0.4215,0.0,0.0,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00
2,Jkj36B1YuDU,ok who else just listens to sad song when ur a...,0.399,0.450,0.151,-0.8271,0.0,0.0,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,intense,0.5 to 1.0
3,Jkj36B1YuDU,same as three,0.000,1.000,0.000,0.0000,0.0,0.0,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00
4,Jkj36B1YuDU,still here at 295 million views 2025🎉 january ...,0.000,1.000,0.000,0.0000,0.0,0.0,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5022,A48hOToMuRE,❤❤❤ this song makes my heart explode.,0.000,1.000,0.000,0.0000,0.0,0.0,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,intense,0.5 to 1.0
5023,A48hOToMuRE,"can anyone explain why people sing w this 1 ""a...",0.000,1.000,0.000,0.0000,0.0,0.0,0.0,1.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,intense,0.5 to 1.0,None,0.00 to 0.00
5024,A48hOToMuRE,this a classic,0.000,1.000,0.000,0.0000,0.0,0.0,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00
5025,A48hOToMuRE,christopher extension,0.000,1.000,0.000,0.0000,0.0,1.0,0.0,0.0,...,None,0.00 to 0.00,intense,0.5 to 1.0,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00


In [54]:
df_sad_comment.to_csv("sad_song_ytcomment.csv",index=False)
df_sad_song.to_csv("sad_song_ytvideo_statistics.csv", index=False)